# 层和块
首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2012, -0.0384, -0.0408, -0.0769, -0.2901, -0.1609,  0.0610, -0.0323,
         -0.0861,  0.3825],
        [-0.1667, -0.0074,  0.1415, -0.0140, -0.2409, -0.1080,  0.0777, -0.0373,
         -0.0189,  0.1632]], grad_fn=<AddmmBackward0>)

nn.Sequential定义了一种特殊的Module

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)  # net()继承Module类，而Module类的call会执行forward

tensor([[-0.1951, -0.1727, -0.2407,  0.0903, -0.1097, -0.1112,  0.3084, -0.2719,
          0.0961,  0.1317],
        [-0.2934, -0.2127, -0.2851,  0.1194, -0.0982, -0.1600,  0.4783, -0.1689,
         -0.0759,  0.2281]], grad_fn=<AddmmBackward0>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1497, -0.1304, -0.0384,  0.1234,  0.0503, -0.3670, -0.1865,  0.1663,
          0.2514, -0.0935],
        [-0.1865, -0.0665, -0.1365,  0.2439,  0.0008, -0.3215, -0.2396,  0.2229,
          0.2753, -0.0448]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


net = FixedHiddenMLP()
net(X)

tensor(0.2058, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))


chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0055, grad_fn=<SumBackward0>)